In [9]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Carregar os dados
file_path_dolar =('C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv')
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])

# Separar os últimos 10 dias de dados
ultimos_10_dias = data_dolar.tail(7)
data_dolar = data_dolar.iloc[:-7]

# Continuar com o pré-processamento como antes...

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino e teste
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2022-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2022-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data','Dif_Preco_Dolar','Preco_Real', 'Dif_Preco_Real','Preco_Dolar'])

y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data','Dif_Preco_Dolar','Preco_Real', 'Dif_Preco_Real','Preco_Dolar'])

# Calcular a média para as colunas e substituir NaN pela média
mean_values_train = X_train.mean()
X_train.fillna(mean_values_train, inplace=True)

mean_values_test = X_test.mean()
X_test.fillna(mean_values_test, inplace=True)

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de regressão linear
model = LinearRegression()
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcular as métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

# Criar um DataFrame com as datas, valores reais e previstos
resultados = pd.DataFrame()
resultados['Data'] = test_data['Data']
resultados['Valor_Real'] = y_test
resultados['Valor_Previsto'] = y_pred

# Exibir os primeiros registros para verificar
print(resultados.head())

# Preparar dados dos últimos 10 dias para a previsão
X_ultimos_10_dias = ultimos_10_dias.drop(columns=['Data','Dif_Preco_Dolar','Preco_Real', 'Dif_Preco_Real','Preco_Dolar'])
X_ultimos_10_dias.fillna(mean_values_train, inplace=True)

# Prever os valores para 'ultimos_10_dias'
y_ultimos_10_dias_pred = model.predict(X_ultimos_10_dias)

# Comparar as previsões com os valores reais
ultimos_10_dias['Valor_Previsto'] = y_ultimos_10_dias_pred
print(ultimos_10_dias[['Data', 'Dif_Preco_Dolar', 'Valor_Previsto']])


ValueError: Found array with 0 sample(s) (shape=(0, 26)) while a minimum of 1 is required by LinearRegression.

O MELHOR MELHOR

In [13]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import lightgbm as lgb
import shap

# Carregar os dados
file_path_dolar =('C:/Users/milen/OneDrive/Documentos/TCC/Bases/euro.csv')
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime e ordenar os dados
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])
# data_dolar = data_dolar.sort_values(by='Data')

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 0.4 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino, teste e validação
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real','Dif_Preco_Dolar'])

y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real','Dif_Preco_Dolar'])

# Calcular a média para as colunas e substituir NaN pela média
mean_values_train = X_train.mean()
X_train.fillna(mean_values_train, inplace=True)

mean_values_test = X_test.mean()
X_test.fillna(mean_values_test, inplace=True)

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de regressão linear
model = LinearRegression()
#model = lgb.LGBMRegressor()
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcular as métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

# Criar um DataFrame com as datas, valores reais e previstos
resultados = pd.DataFrame()
resultados['Data'] = test_data['Data']
resultados['Valor_Real'] = y_test
resultados['Valor_Previsto'] = y_pred

# Exibir os primeiros registros para verificar
print(resultados.head(20))


MSE: 1.4144627386124147
MAE: 0.93425207546037
R^2: 0.4184378432547554
           Data  Valor_Real  Valor_Previsto
4192 2020-01-03       -1.07       -0.282553
4194 2020-01-07       -0.83        0.409178
4196 2020-01-09       -2.08       -0.173142
4197 2020-01-10        1.16        0.091813
4199 2020-01-14       -0.44        0.481151
4200 2020-01-15       -1.68       -0.403290
4201 2020-01-16       -0.97        0.164825
4202 2020-01-17        1.16        0.844743
4203 2020-01-21       -1.00       -0.038981
4204 2020-01-22        1.19        0.609236
4205 2020-01-23        0.52        0.637602
4206 2020-01-24       -1.34        0.202053
4208 2020-01-28        0.53        0.372333
4209 2020-01-29       -1.92       -0.230156
4210 2020-01-30       -1.22        0.025656
4211 2020-01-31        0.28       -0.604892
4213 2020-02-04       -0.38        0.156067
4214 2020-02-05        1.04        0.758674
4215 2020-02-06       -0.19       -0.326385
4216 2020-02-07       -0.42       -0.087336


In [ ]:
# ... [seu código anterior permanece inalterado até o treinamento do modelo] ...

# Definindo o tamanho da janela deslizante
tamanho_da_janela = 5

# Inicializando o DataFrame de resultados
resultados = pd.DataFrame(columns=['Data', 'Valor_Real', 'Valor_Previsto'])

# Fazendo previsões em uma janela deslizante
for i in range(0, len(test_data), tamanho_da_janela):
    # Selecionar os dados da janela atual
    X_test_janela = X_test.iloc[i:i+tamanho_da_janela]
    y_test_janela = y_test.iloc[i:i+tamanho_da_janela]
    datas_janela = test_data['Data'].iloc[i:i+tamanho_da_janela]

    # Prever os valores para a janela atual
    y_pred_janela = model.predict(X_test_janela)

    # Armazenar os resultados
    janela_resultados = pd.DataFrame({
        'Data': datas_janela,
        'Valor_Real': y_test_janela,
        'Valor_Previsto': y_pred_janela
    })
    resultados = resultados.append(janela_resultados, ignore_index=True)

# Calcular as métricas de desempenho para todo o período de teste
mse = mean_squared_error(resultados['Valor_Real'], resultados['Valor_Previsto'])
mae = mean_absolute_error(resultados['Valor_Real'], resultados['Valor_Previsto'])
r2 = r2_score(resultados['Valor_Real'], resultados['Valor_Previsto'])

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)
print(resultados.head(20))


In [269]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import lightgbm as lgb
import shap
import xgboost as xgb

# Carregar os dados
file_path_dolar =('C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima_atualizado.csv')
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime e ordenar os dados
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])
# data_dolar = data_dolar.sort_values(by='Data')

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 0.4 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino, teste e validação
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic','OperacoesSeleic','UltimoEUR', 'Preco_Dolar', 'Preco_Real','Dif_Preco_Real'])

y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data','Pontos', 'Dif_Preco_Dolar','TaxaSelic','OperacoesSeleic','UltimoEUR', 'Preco_Dolar', 'Preco_Real','Dif_Preco_Real'])

# Calcular a média para as colunas e substituir NaN pela média
mean_values_train = X_train.mean()
X_train.fillna(mean_values_train, inplace=True)

mean_values_test = X_test.mean()
X_test.fillna(mean_values_test, inplace=True)

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de regressão linear
model = xgb.XGBRegressor(objective ='reg:squarederror')
#model = lgb.LGBMRegressor()
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcular as métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

# Criar um DataFrame com as datas, valores reais e previstos
resultados = pd.DataFrame()
resultados['Data'] = test_data['Data']
resultados['Valor_Real'] = y_test
resultados['Valor_Previsto'] = y_pred

# Exibir os primeiros registros para verificar
print(resultados.head())


MSE: 3.1077767802435763
MAE: 1.4787053674773167
R^2: -1.2523159767585392
           Data  Valor_Real  Valor_Previsto
6391 2023-06-26        0.45        1.877941
6392 2023-06-27        0.10        1.564456
6395 2023-06-30        1.30        1.702488
6396 2023-07-03       -0.63        1.720908
6399 2023-07-06       -1.34        1.938566


In [185]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Carregar os dados
file_path_dolar = 'C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima_atualizado.csv'
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime e definir como índice
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])
data_dolar = data_dolar.set_index('Data')

# Identificar e tratar datas duplicadas (mantendo a primeira ocorrência)
data_dolar = data_dolar[~data_dolar.index.duplicated(keep='first')]

# Redefinir a frequência para diária
data_dolar = data_dolar.asfreq('D')

# Tratar valores faltantes (por exemplo, preenchendo com o valor anterior)
data_dolar = data_dolar.fillna(method='ffill')

# Selecionar a coluna de interesse para a análise ARIMA
y = data_dolar['Dif_Preco_Dolar']

# Dividir os dados em treino e teste
train_end = pd.to_datetime("2020-01-01")
y_train = y[y.index < train_end]
y_test = y[y.index >= train_end]

# Definir e ajustar o modelo ARIMA
p = 1  # Autoregressivo
d = 1  # Diferenciação
q = 1  # Média móvel
model = ARIMA(y_train, order=(p, d, q))
model_fit = model.fit()

# Fazer previsões
y_pred = model_fit.forecast(steps=len(y_test))

# Calcular métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)



MSE: 11.64856444442459
MAE: 2.5378110375356426
R^2: 6.189585840199463e-05


In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Carregar os dados
file_path_dolar = 'C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima_cme.csv'
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime e ordenar os dados
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 0.4 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino e teste
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Normalizar os dados
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Redimensionar os dados para o formato [samples, time_steps, features]
X_train_scaled = np.reshape(X_train_scaled, (X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_scaled = np.reshape(X_test_scaled, (X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Construir o modelo LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(1))

# Compilar o modelo
model.compile(loss='mean_squared_error', optimizer='adam')

# Treinar o modelo
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=2)

# Fazer previsões
y_pred = model.predict(X_test_scaled)

# Calcular as métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

# Criar um DataFrame com as datas, valores reais e previstos
resultados = pd.DataFrame()
resultados['Data'] = test_data['Data']
resultados['Valor_Real'] = y_test
resultados['Valor_Previsto'] = y_pred.flatten()  # Achatando a saída para combinar com o formato do y_test

# Exibir os primeiros registros para verificar
print(resultados.head())


Epoch 1/100
43/43 - 2s - loss: 3.2658 - 2s/epoch - 53ms/step
Epoch 2/100
43/43 - 0s - loss: 3.1321 - 69ms/epoch - 2ms/step
Epoch 3/100
43/43 - 0s - loss: 3.0891 - 59ms/epoch - 1ms/step
Epoch 4/100
43/43 - 0s - loss: 3.0767 - 69ms/epoch - 2ms/step
Epoch 5/100
43/43 - 0s - loss: 3.0652 - 71ms/epoch - 2ms/step
Epoch 6/100
43/43 - 0s - loss: 3.0642 - 67ms/epoch - 2ms/step
Epoch 7/100
43/43 - 0s - loss: 3.0611 - 60ms/epoch - 1ms/step
Epoch 8/100
43/43 - 0s - loss: 3.0644 - 75ms/epoch - 2ms/step
Epoch 9/100
43/43 - 0s - loss: 3.0531 - 68ms/epoch - 2ms/step
Epoch 10/100
43/43 - 0s - loss: 3.0467 - 72ms/epoch - 2ms/step
Epoch 11/100
43/43 - 0s - loss: 3.0477 - 72ms/epoch - 2ms/step
Epoch 12/100
43/43 - 0s - loss: 3.0499 - 65ms/epoch - 2ms/step
Epoch 13/100
43/43 - 0s - loss: 3.0457 - 63ms/epoch - 1ms/step
Epoch 14/100
43/43 - 0s - loss: 3.0424 - 62ms/epoch - 1ms/step
Epoch 15/100
43/43 - 0s - loss: 3.0357 - 61ms/epoch - 1ms/step
Epoch 16/100
43/43 - 0s - loss: 3.0344 - 63ms/epoch - 1ms/step
Ep

In [268]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Carregar os dados
file_path_dolar = 'C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima_atualizado.csv'
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime e ordenar os dados
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 0.4 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino e teste
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

# Calcular a média para as colunas e substituir NaN pela média
mean_values_train = X_train.mean()
X_train.fillna(mean_values_train, inplace=True)

mean_values_test = X_test.mean()
X_test.fillna(mean_values_test, inplace=True)

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de regressão linear
model = LinearRegression()
model.fit(X_train, y_train)

# Definindo o tamanho da janela deslizante, por exemplo, 5 dias
tamanho_da_janela = 30

# Inicializando o DataFrame de resultados
resultados = pd.DataFrame()

# Fazendo previsões em uma janela deslizante
for i in range(0, len(test_data), tamanho_da_janela):
    # Selecionar os dados da janela atual
    X_test_janela = X_test.iloc[i:i+tamanho_da_janela]
    y_test_janela = y_test.iloc[i:i+tamanho_da_janela]
    datas_janela = test_data['Data'].iloc[i:i+tamanho_da_janela]

    # Prever os valores para a janela atual
    y_pred_janela = model.predict(X_test_janela)

    # Armazenar os resultados
    janela_resultados = pd.DataFrame({
        'Data': datas_janela,
        'Valor_Real': y_test_janela,
        'Valor_Previsto': y_pred_janela
    })
    resultados = pd.concat([resultados, janela_resultados], ignore_index=True)

# Calcular as métricas de desempenho para todo o período de teste
mse = mean_squared_error(resultados['Valor_Real'], resultados['Valor_Previsto'])
mae = mean_absolute_error(resultados['Valor_Real'], resultados['Valor_Previsto'])
r2 = r2_score(resultados['Valor_Real'], resultados['Valor_Previsto'])

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)
print(resultados.head(20))


MSE: 0.9275394800923671
MAE: 0.839113278053472
R^2: 0.3277792654327605
         Data  Valor_Real  Valor_Previsto
0  2023-06-26        0.45        1.403968
1  2023-06-27        0.10       -1.003845
2  2023-06-30        1.30        2.252853
3  2023-07-03       -0.63       -0.094444
4  2023-07-06       -1.34       -2.022010
5  2023-07-13        1.85        0.812661
6  2023-07-18       -0.39        0.288551
7  2023-07-21        2.79        1.164435
8  2023-07-24        2.62        1.912676
9  2023-07-25       -0.15       -1.257611
10 2023-07-26        0.39        1.037881
11 2023-07-27       -0.67        0.535776
12 2023-08-02        0.03       -0.073454
13 2023-08-11       -1.40        0.302492
14 2023-08-16        0.19        0.523376
15 2023-08-18        0.25       -0.571402
16 2023-08-21        0.32       -0.066038
17 2023-08-22        1.15        1.526829
18 2023-08-28        1.29       -0.063466
19 2023-09-04        0.78        0.829645


In [273]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Carregar os dados
file_path_dolar = 'C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima_atualizado.csv'
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.2 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino e teste
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

# Calcular a média para as colunas e substituir NaN pela média
mean_values_train = X_train.mean()
X_train.fillna(mean_values_train, inplace=True)

mean_values_test = X_test.mean()
X_test.fillna(mean_values_test, inplace=True)

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de Random Forest
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcular as métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

# Criar um DataFrame com as datas, valores reais e previstos
resultados = pd.DataFrame()
resultados['Data'] = test_data['Data']
resultados['Valor_Real'] = y_test
resultados['Valor_Previsto'] = y_pred

# Exibir os primeiros registros para verificar
print(resultados.head())


MSE: 4.774370601507938
MAE: 1.8219976190476193
R^2: -0.8375899191562914
           Data  Valor_Real  Valor_Previsto
5527 2020-01-07       -0.83         -0.0092
5529 2020-01-09       -2.08          0.3886
5530 2020-01-10        1.16         -0.0817
5531 2020-01-13       -3.78          0.2513
5532 2020-01-14       -0.44          0.3201


In [275]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Carregar os dados
file_path_dolar = 'C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima_atualizado.csv'
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.2 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino e teste
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])
y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

# Calcular a média para as colunas e substituir NaN pela média
mean_values_train = X_train.mean()
X_train.fillna(mean_values_train, inplace=True)
mean_values_test = X_test.mean()
X_test.fillna(mean_values_test, inplace=True)

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de Random Forest
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Definindo o tamanho da janela deslizante, por exemplo, 5 dias
tamanho_da_janela = 1

# Inicializando o DataFrame de resultados
resultados = pd.DataFrame()

# Fazendo previsões em uma janela deslizante
for i in range(0, len(test_data), tamanho_da_janela):
    # Selecionar os dados da janela atual
    X_test_janela = X_test.iloc[i:i+tamanho_da_janela]
    y_test_janela = y_test.iloc[i:i+tamanho_da_janela]
    datas_janela = test_data['Data'].iloc[i:i+tamanho_da_janela]

    # Prever os valores para a janela atual
    y_pred_janela = model.predict(X_test_janela)

    # Armazenar os resultados
    janela_resultados = pd.DataFrame({
        'Data': datas_janela,
        'Valor_Real': y_test_janela,
        'Valor_Previsto': y_pred_janela
    })
    resultados = pd.concat([resultados, janela_resultados], ignore_index=True)

# Calcular as métricas de desempenho para todo o período de teste
mse = mean_squared_error(resultados['Valor_Real'], resultados['Valor_Previsto'])
mae = mean_absolute_error(resultados['Valor_Real'], resultados['Valor_Previsto'])
r2 = r2_score(resultados['Valor_Real'], resultados['Valor_Previsto'])

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)
print(resultados.head(20))


MSE: 3.5576828395238103
MAE: 1.5515880952380956
R^2: -0.36930344690867933
         Data  Valor_Real  Valor_Previsto
0  2020-01-07       -0.83          0.5165
1  2020-01-09       -2.08          0.1039
2  2020-01-10        1.16          0.1843
3  2020-01-13       -3.78          0.0860
4  2020-01-14       -0.44          0.2536
5  2020-01-15       -1.68         -0.0753
6  2020-01-16       -0.97         -0.2445
7  2020-01-17        1.16          0.0380
8  2020-01-20       -0.93         -0.3680
9  2020-01-21       -1.00         -0.2129
10 2020-01-22        1.19          0.0662
11 2020-01-23        0.52         -0.1438
12 2020-01-24       -1.34         -0.2713
13 2020-01-27       -2.92         -0.3079
14 2020-01-28        0.53         -0.0642
15 2020-01-29       -1.92         -0.4634
16 2020-01-30       -1.22         -0.2677
17 2020-01-31        0.28         -0.2239
18 2020-02-06       -0.19         -0.0974
19 2020-02-07       -0.42          0.0499


In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Carregar os dados
file_path_dolar = 'C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima_atualizado.csv'
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 0.4 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino e teste
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])
y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data', 'Pontos', 'Dif_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real'])

# Calcular a média para as colunas e substituir NaN pela média
mean_values_train = X_train.mean()
X_train.fillna(mean_values_train, inplace=True)
mean_values_test = X_test.mean()
X_test.fillna(mean_values_test, inplace=True)

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de Random Forest
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Definindo o tamanho da janela deslizante, por exemplo, 5 dias
tamanho_da_janela = 30

# Inicializando o DataFrame de resultados
resultados = pd.DataFrame()

# Fazendo previsões em uma janela deslizante
for i in range(0, len(test_data), tamanho_da_janela):
    # Selecionar os dados da janela atual
    X_test_janela = X_test.iloc[i:i+tamanho_da_janela]
    y_test_janela = y_test.iloc[i:i+tamanho_da_janela]
    datas_janela = test_data['Data'].iloc[i:i+tamanho_da_janela]

    # Prever os valores para a janela atual
    y_pred_janela = model.predict(X_test_janela)

    # Armazenar os resultados
    janela_resultados = pd.DataFrame({
        'Data': datas_janela,
        'Valor_Real': y_test_janela,
        'Valor_Previsto': y_pred_janela
    })
    resultados = pd.concat([resultados, janela_resultados], ignore_index=True)

# Calcular as métricas de desempenho para todo o período de teste
mse = mean_squared_error(resultados['Valor_Real'], resultados['Valor_Previsto'])
mae = mean_absolute_error(resultados['Valor_Real'], resultados['Valor_Previsto'])
r2 = r2_score(resultados['Valor_Real'], resultados['Valor_Previsto'])

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)
print(resultados.head(20))


Epoch 1/100
115/115 - 4s - loss: 0.0050 - 4s/epoch - 33ms/step
Epoch 2/100
115/115 - 1s - loss: 6.7618e-04 - 1s/epoch - 10ms/step
Epoch 3/100
115/115 - 1s - loss: 6.6512e-04 - 1s/epoch - 9ms/step
Epoch 4/100
115/115 - 1s - loss: 6.7020e-04 - 1s/epoch - 9ms/step
Epoch 5/100
115/115 - 1s - loss: 6.5666e-04 - 1s/epoch - 9ms/step
Epoch 6/100
115/115 - 1s - loss: 5.9453e-04 - 1s/epoch - 10ms/step
Epoch 7/100
115/115 - 1s - loss: 5.8878e-04 - 1s/epoch - 9ms/step
Epoch 8/100
115/115 - 1s - loss: 5.4854e-04 - 1s/epoch - 10ms/step
Epoch 9/100
115/115 - 1s - loss: 5.5160e-04 - 1s/epoch - 10ms/step
Epoch 10/100
115/115 - 1s - loss: 4.7383e-04 - 1s/epoch - 10ms/step
Epoch 11/100
115/115 - 1s - loss: 4.8700e-04 - 1s/epoch - 10ms/step
Epoch 12/100
115/115 - 1s - loss: 4.9810e-04 - 1s/epoch - 10ms/step
Epoch 13/100
115/115 - 1s - loss: 4.1781e-04 - 1s/epoch - 10ms/step
Epoch 14/100
115/115 - 1s - loss: 3.9129e-04 - 1s/epoch - 10ms/step
Epoch 15/100
115/115 - 1s - loss: 4.1969e-04 - 1s/epoch - 9ms/ste